# Topic Modelling using Latent Dirichlet Allocation (LDA)
Major assumptions:
1. Each document is as homogeneous as possible in terms of topic, i.e. each document is likely expplained by as less topics as possible
2. Each word is as homogeneous as possible in terms of a topic, i.e. each word is as unique as possible for a topic description.

## Load necessary libraries for this notebook

In [1]:
# Download nltk stowards for stopword removal

import nltk
nltk.download("stopwords")

# LDA visualization using pyLDAvis

!pip install pyLDAvis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 13.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=efbc0a86f71f08f49d35e99a9ab004c1aba0501cd177749b73bcb82075abbcfd
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=1808111400537f6e1e90f506b02409394fbec7ef2d7a6cd8d06334ed9c915294
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pyLDAvis sklearn


In [2]:
# synatctic processing related libraries
from nltk.corpus import stopwords
import spacy
import re

# Numpy and pandas to read the data
import numpy as np
import pandas as pd
from pandas.core.base import value_counts

# LDA from gensim

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Visulaization
import pyLDAvis
import pyLDAvis.gensim_models

# Stop showing deprecation realted warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


# Load and pre-process data

In [3]:
bc_incl = pd.read_csv('/content/BreastCancer_incl.csv',index_col=[0], encoding="utf-8")
print("Number of protocols in the file", bc_incl['Trial_ID'].nunique())
print("Total number of inclusion criteria in the file", bc_incl.shape[0])

Number of protocols in the file 213
Total number of inclusion criteria in the file 1979


In [4]:
# Check for missing values related to encodings
bc_incl[bc_incl.isnull().any(axis=1)]

,Trial_ID,Incl_crit


There are no missing records.

### Pre-process
- No cleaning
- Simple cleaning : remove alpha numeric letters and convert to lower case 
- Super cleaning:
  - Simple cleaning
  - Remove stopwords
  - Create pure words using lemmatization

**No cleaning and simple cleaning**

In [5]:
no_clean= list(bc_incl['Incl_crit'])
simple_clean = [re.sub(r'[^\w]',' ',sent.lower()) for sent in bc_incl['Incl_crit']]

In [6]:
#New
import string

test_clean = [sent.translate(str.maketrans('', '', string.punctuation)).lower() for sent in bc_incl['Incl_crit']]

In [7]:
print('Oringinal text\n', bc_incl['Incl_crit'][7])
print('Clended text after removing alphanumeric symbols and making lower case\n', simple_clean[7])

Oringinal text
 TNBC patients （ HER2-neu 0-1+ by IHC or FISH-negative by ASCO CAP guidelines）
Clended text after removing alphanumeric symbols and making lower case
 tnbc patients   her2 neu 0 1  by ihc or fish negative by asco cap guidelines 


Note that, we might well use data without cleaning because 
1. TNBC, FISH these are short form of some sort of diagnosis. After cleasing the meaning is changed (e.g. fish might be interpreted as an creature rather than a test type)

**Super Cleaning**

In [8]:
# Stopword removal
stopwords= stopwords.words("english")

In [9]:
# POS_tag specific lemmatization
def lemmatize_text(texts, lemma_tags=["NOUN","ADJ", "VERB", "ADV"]):
  # Load the spacy model and remove NER or Parsing pipelines to make it faster
  nlp= spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out=[]
  for text in texts:
    sent=nlp(text)
    new_text = []
    for token in sent:
      if token.pos_ in lemma_tags:
        new_text.append(token.lemma_)
    final= " ".join(new_text)
    texts_out.append(final)
  return texts_out

In [10]:
lemmatized_texts= lemmatize_text(simple_clean)

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [11]:
print('Oringinal text\n', no_clean[7])
print('Clended text after removing alphanumeric symbols and making lower case\n', simple_clean[7])
print('Lemmatized version\n', lemmatized_texts[7])

Oringinal text
 TNBC patients （ HER2-neu 0-1+ by IHC or FISH-negative by ASCO CAP guidelines）
Clended text after removing alphanumeric symbols and making lower case
 tnbc patients   her2 neu 0 1  by ihc or fish negative by asco cap guidelines 
Lemmatized version
 tnbc patient fish negative guideline


Lemmatization can pose potential dangerous threat as it has removed key information like `ASCO CAP`

In [12]:
# Stopwords removal using gensim utils pre processing
def gen_words(texts):
  final=[]
  for text in texts:
    # Along with pre processing the deaccenct option allows to remove non english letters
    new= gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return final

In [13]:
super_clean= gen_words(lemmatized_texts)

In [14]:
print('Oringinal text\n', no_clean[7])
print('Clended text after removing alphanumeric symbols and making lower case\n', simple_clean[7])
print('Super clean version\n', super_clean[7])

Oringinal text
 TNBC patients （ HER2-neu 0-1+ by IHC or FISH-negative by ASCO CAP guidelines）
Clended text after removing alphanumeric symbols and making lower case
 tnbc patients   her2 neu 0 1  by ihc or fish negative by asco cap guidelines 
Super clean version
 ['tnbc', 'patient', 'fish', 'negative', 'guideline']



Idea to improve further:
1. Use no_clean version with stop word removal only.
2. Use simple clean with stop word removal
3. Try using bi-gram or trigram to keep expression like 'FISH-positive' as it is

For this notebook we are working only with super clean version.

**Data dictionary**

this step will create a bag of words representation i.e. unique words and it's frequency in each document. 
*corpora.Dictionary()* a mapping between words and their integer ids <br>
*corpora.Dictionary.doc2bow()* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document;

In [15]:
#a mapping between words and their integer ids
id2word= corpora.Dictionary(super_clean)

corpus = []

for text in super_clean:
  #Bag of words representation
  new= id2word.doc2bow(text)
  corpus.append(new)


Demonstration

In [16]:
print('Super clean version\n', super_clean[101])

Super clean version
 ['receptor', 'status', 'receptor', 'progesterone', 'receptor', 'positive', 'human', 'epidermal', 'growth', 'factor', 'receptor', 'negative']


In [17]:
print('All the words in the complete set of texts (i.e all senteces) are converted to an unique ID\n')
print('Frequency of each ID in each sentence is counted. The final representation is a list of tuples (ID, frequency) for each sentence')

print(corpus[101])

All the words in the complete set of texts (i.e all senteces) are converted to an unique ID

Frequency of each ID in each sentence is counted. The final representation is a list of tuples (ID, frequency) for each sentence
[(25, 1), (28, 1), (42, 1), (387, 1), (388, 1), (389, 1), (390, 1), (391, 1), (392, 4)]


In [18]:
print('We can see that the word receptor appear 4 times')
print('According to the logic stated above, the word RECEPTOR should map to ID 392')
print('word map for ID 392:', id2word[392])
print('Great job !!')

We can see that the word receptor appear 4 times
According to the logic stated above, the word RECEPTOR should map to ID 392
word map for ID 392: receptor
Great job !!


## LDA model

Initial run with super clean data. <br>
- Number of topics (num_topics): 30
- Seed (random_state): 2022
- update parameters (update_every) after every epoch (i.e., 1)
- Number of senetences to consider (chunksize) at each run (i.e. batch size): 128
- Number of epochs (passes): 30

In [19]:
num_topics=30

lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=2022, update_every=1, chunksize=128, passes=30, alpha='auto')

## Visualizing Data

In [20]:
#enable automatic D3 display of prepared model data in the IPython notebook
pyLDAvis.enable_notebook()

`pyLDAvis.gensim_models.prepare` parameters:
- R: Number of words to display per topic
- mds: Multidimensional scaling 
  - PCA
  - t-SNE

In [21]:
vis_pca = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, R=20, mds='pcoa')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [22]:
vis_pca

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4     -0.047703  0.109050       1        1  6.216758
19     0.269662  0.126023       2        1  5.687644
12    -0.029175 -0.013038       3        1  5.560307
22     0.003006  0.081625       4        1  5.194096
24     0.080758  0.189881       5        1  4.900039
28    -0.086856  0.197320       6        1  4.573786
9      0.186441 -0.023762       7        1  4.387309
14    -0.074755 -0.136704       8        1  4.251210
15    -0.083361  0.205656       9        1  4.068867
6     -0.119115 -0.056043      10        1  4.067749
16    -0.085388  0.000680      11        1  3.711855
5     -0.054561 -0.044493      12        1  3.582446
11    -0.032835  0.027074      13        1  3.551962
23    -0.086014 -0.022320      14        1  3.505185
0      0.251210 -0.084062      15        1  3.484973
17    -0.080286  0.076812      16        1  3.391288
25    -0.025489  0.051553      17        1  2.846826
2     -0.005268 -0.037549      18        1  2.785239
27    -0.051450  0.032013      19        1  2.554038
18    -0.027866 -0.019486      20        1  2.496883
26     0.176491 -0.027341      21        1  2.376974
8     -0.076222 -0.133202      22        1  2.269189
20     0.080105 -0.100745      23        1  2.126840
3     -0.032566 -0.066137      24        1  1.954877
21    -0.026996 -0.033318      25        1  1.895512
29    -0.038848 -0.070842      26        1  1.883752
13    -0.040381 -0.083766      27        1  1.861031
1      0.105096 -0.051755      28        1  1.804129
7     -0.021044 -0.054531      29        1  1.629702
10    -0.026590 -0.038594      30        1  1.379536, topic_info=               Term        Freq       Total Category  logprob  loglift
26          patient  455.000000  455.000000  Default  20.0000  20.0000
3            breast  354.000000  354.000000  Default  19.0000  19.0000
4            cancer  288.000000  288.000000  Default  18.0000  18.0000
148       treatment  230.000000  230.000000  Default  17.0000  17.0000
2              year  153.000000  153.000000  Default  16.0000  16.0000
...             ...         ...         ...      ...      ...      ...
1903        propose    3.258519    4.175961  Topic30  -4.3832   4.0354
1027      technique    3.159794    4.077236  Topic30  -4.4140   4.0285
1238    granulocyte    3.084551    4.001994  Topic30  -4.4381   4.0230
952   hematological    2.910567    3.828010  Topic30  -4.4962   4.0094
1024     accurately    2.745747    3.663189  Topic30  -4.5544   3.9951

[693 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
233      13  0.985385   ability
236      24  0.975144      able
1023     14  0.826352     above
109      16  0.899441   absence
50       30  0.968881  absolute
...     ...       ...       ...
107      23  0.552683     woman
453      19  0.723660      work
93       13  0.984592     write
2        20  0.991912      year
736      20  0.635147        yr

[694 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 20, 13, 23, 25, 29, 10, 15, 16, 7, 17, 6, 12, 24, 1, 18, 26, 3, 28, 19, 27, 9, 21, 4, 22, 30, 14, 2, 8, 11])

Few topics are well seperated , but still there is overlap between topics.
Let's try 2D mapping with t-SNE.

In [23]:
vis_tsne = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, R=20, mds='tsne')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by defau

In [24]:
vis_tsne

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
4      119.070190   64.655502       1        1  6.216748
19      54.382130  109.799164       2        1  5.673934
12    -161.423965 -103.744957       3        1  5.560301
22    -135.370758   36.673065       4        1  5.194139
24     177.317215  -66.872452       5        1  4.909779
28     -16.436356  142.453796       6        1  4.573798
9       31.052383  204.777573       7        1  4.387312
14     -25.024384 -186.083466       8        1  4.251191
15     -68.157486   20.934038       9        1  4.068950
6       -3.751695    4.961509      10        1  4.067691
16    -114.470009 -179.605469      11        1  3.708967
5     -110.729126  -40.476776      12        1  3.582465
11      42.552418  -42.789406      13        1  3.551960
23     -10.661536   71.922211      14        1  3.505228
0      112.652969  -16.279951      15        1  3.485465
17    -195.499146  -25.853945      16        1  3.391223
25    -207.165924   66.070419      17        1  2.849635
2       10.217649 -110.752060      18        1  2.785273
27     -81.255081   96.477776      19        1  2.554078
18     -77.253441  192.755219      20        1  2.496887
26     182.353027  110.412880      21        1  2.380297
8       89.766510  -93.997673      22        1  2.269218
20     -36.483528  -52.542774      23        1  2.126884
3      -72.052170 -112.144821      24        1  1.954878
21     144.822769 -150.811508      25        1  1.895500
29    -148.475357  135.678162      26        1  1.883763
13     189.811615   17.382956      27        1  1.861032
1       56.555355   33.755768      28        1  1.804172
7       58.988537 -178.376541      29        1  1.629695
10     110.556717  164.236694      30        1  1.379536, topic_info=               Term        Freq       Total Category  logprob  loglift
26          patient  455.000000  455.000000  Default  20.0000  20.0000
3            breast  354.000000  354.000000  Default  19.0000  19.0000
4            cancer  288.000000  288.000000  Default  18.0000  18.0000
148       treatment  229.000000  229.000000  Default  17.0000  17.0000
2              year  153.000000  153.000000  Default  16.0000  16.0000
...             ...         ...         ...      ...      ...      ...
1903        propose    3.258518    4.175990  Topic30  -4.3832   4.0353
1027      technique    3.159794    4.077265  Topic30  -4.4140   4.0285
1238    granulocyte    3.084551    4.002023  Topic30  -4.4381   4.0230
952   hematological    2.910567    3.828039  Topic30  -4.4962   4.0094
1024     accurately    2.745747    3.663218  Topic30  -4.5544   3.9951

[693 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
233      13  0.985385   ability
236      24  0.975142      able
1023     14  0.826340     above
109      16  0.899454   absence
50       30  0.968880  absolute
...     ...       ...       ...
107      23  0.552650     woman
453      19  0.723646      work
93       13  0.984592     write
2        20  0.991910      year
736      20  0.635141        yr

[694 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 20, 13, 23, 25, 29, 10, 15, 16, 7, 17, 6, 12, 24, 1, 18, 26, 3, 28, 19, 27, 9, 21, 4, 22, 30, 14, 2, 8, 11])

### Extract all topic descriptions

In [36]:
#Store topic descriptions in a dictionary
topic_dict={'ID': list(), 'Words':list()}

for topicid in range(num_topics):
  topic_desc=gensim.models.LdaModel.get_topic_terms(lda_model,topicid=topicid,topn=20)
  topic_word=[id2word[x[0]] for x in topic_desc]
  topic_dict['ID'].append(topicid)
  topic_dict['Words'].append(topic_word)

In [77]:
#onvert results to a data frame
pd.DataFrame.from_dict(topic_dict)

,ID,Words
0,0,"[study, woman, contraception, undergo, entry, ..."
1,1,"[willing, control, protocol, comply, birth, st..."
2,2,"[platelet, treat, radiation, grade, provide, p..."
3,3,"[history, able, procedure, active, other, rela..."
4,4,"[breast, cancer, confirm, stage, invasive, his..."
5,5,"[least, month, at, trial, life, surgical, expe..."
6,6,"[eligible, creatinine, cell, cytologically, re..."
7,7,"[lymph, node, less, currently, physical, axill..."
8,8,"[negative, positive, receptor, progesterone, h..."
9,9,"[use, potential, childbeare, method, agree, bl..."


Demonstration of function `get_topic_terms` <br>
Return a list of (word_id, probability) 2-tuples for the most probable words in topic topicid.

Only return 2-tuples for the topn most probable words.


In [79]:
print('Topic description of 1st topic with top 20 words\n ',gensim.models.LdaModel.get_topic_terms(lda_model,topicid=1,topn=20))

Topic description of 1st topic with top 20 words
  [(262, 0.12935804), (293, 0.10291982), (321, 0.07453537), (528, 0.067721136), (354, 0.062605284), (89, 0.05939475), (319, 0.058500625), (363, 0.029915432), (1913, 0.029367482), (1912, 0.029367482), (318, 0.023051871), (247, 0.016041677), (357, 0.015150953), (361, 0.012715697), (597, 0.012375045), (356, 0.011786299), (337, 0.010371798), (362, 0.010063686), (94, 0.009644796), (1772, 0.009384186)]


Note that, each word is a word_id and the actual word needs to be extracted from id2word dictionary

### Identify top 3 topic for each inclusion criteria

In [73]:
# Top 3 topics for each inclusion criteria
top1=list()
top2=list()
top3=list()

for bow in corpus:
  topic_dist = gensim.models.LdaModel.get_document_topics(lda_model,bow=bow,minimum_probability=1e-3)
  # Sorted by probability of topic 
  topic_dist= sorted(topic_dist,key=lambda tup: tup[1],reverse=True)
  top1.append(topic_dist[0])
  top2.append(topic_dist[1])
  top3.append(topic_dist[2])


Note that, `get_document_topics` provides list of topics (topic_id, probability) that are higher than minimum_probability threshold

In [74]:
# store in a data frame
topic_df= pd.DataFrame(zip(top1, top2, top3), columns=['Topic1','Topic2', 'Topic3'])

In [75]:
# merge with original inclusion criteria file
lda_bc_result = bc_incl.join(topic_df)

In [76]:
lda_bc_result.head()

,Trial_ID,Incl_crit,Topic1,Topic2,Topic3
0,NCT05376241,Female,"(14, 0.22667772)","(22, 0.05178042)","(4, 0.050488904)"
1,NCT05376241,Between 39-49 years of age,"(18, 0.3489918)","(22, 0.043348685)","(4, 0.042267475)"
2,NCT05376241,No history of breast cancer,"(4, 0.31641614)","(3, 0.15250365)","(22, 0.037278406)"
3,NCT05376241,No known BRCA 1/2 mutation,"(22, 0.45737976)","(4, 0.036348604)","(19, 0.034764476)"
4,NCT05600257,individuals were diagnosed with breast cancer ...,"(27, 0.38971403)","(4, 0.24804328)","(28, 0.092094205)"


Most of the inclusion criteria are described by 1 or 2 topics

Demonstration of `get_document_topics`

In [83]:
topic_dist = gensim.models.LdaModel.get_document_topics(lda_model,bow=corpus[10],minimum_probability=1e-3)

A small threshold (minimum_probability) is chosen to ensure that we always get at least 3 top topics per criteria. 
The input text must be in a BOW (bag of words) representation in order to perform topic prediction.

In [84]:
# Sorted by probability - descending order
topic_dist= sorted(topic_dist,key=lambda tup: tup[1],reverse=True)

In [86]:
print('Oringinal text\n', no_clean[10])

print('Lemmatized final version of the text\n', super_clean[10])

# top 3 topics
topic_dist[:3]

Oringinal text
 Any nodal status
Lemmatized final version of the text
 ['nodal', 'status']


[(6, 0.18787658), (29, 0.1785286), (22, 0.043348685)]

Not bad, clusters are well seperated using tsne representation. But do they really have meaning?
Let us do the following:
1. Extract words explaining each topic
2. Map topic IDs to original inclusion criteria to study them better

Reference (implementation):
1. [Topic modelling part 1](https://www.youtube.com/watch?v=TKjjlp5_r7o)
2. [Topic Modelling part 2](https://www.youtube.com/watch?v=UEn3xHNBXJU)
3. [Topic modelling part 3](https://www.youtube.com/watch?v=i74DVqMsRWY)

Reference (Concept):
1. [LDA part 1](https://www.youtube.com/watch?v=T05t-SqKArY)
2. [LDA part 2](https://www.youtube.com/watch?v=BaM1uiCpj_E)
